In [76]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('yellow_taxi_2024_10') \
    .getOrCreate()

## Question 1

In [77]:
spark.version

'3.3.2'

## Question 2

In [78]:
df_raw = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [79]:
df.rdd.getNumPartitions()

16

In [80]:
df_part_4 = df.coalesce(4)

In [81]:
df_part_4.write.mode("overwrite").parquet("data/yellow")

In [82]:
from pyspark.sql import functions as F

In [83]:
yellow_trips = df_part_4 \
	.withColumn('pickup_date', F.to_date(df_part_4.tpep_pickup_datetime)) \
	.withColumn('dropoff_date', F.to_date(df_part_4.tpep_dropoff_datetime))

In [84]:
yellow_trips.createOrReplaceTempView('yellow_2024')

In [85]:
spark.sql("""
SELECT
    count(*)
FROM
    yellow_2024
WHERE
    tpep_pickup_datetime BETWEEN '2024-10-15 00:00:00' AND '2024-10-15 23:59:59'
""").show()

+--------+
|count(1)|
+--------+
|  128909|
+--------+



### Check count using pandas
- slight discrepancy (unclear if difference in timestamp handling?)

In [75]:
df_pd = pd.read_parquet('yellow_tripdata_2024-10.parquet')

In [51]:
df_pd['pickup_date'] = df_pd['tpep_pickup_datetime'].dt.date

oct_15_records = df_pd[df_pd['pickup_date'] == pd.to_datetime('2024-10-15').date()]

print(len(oct_15_records))

128893


## Question 3

In [54]:
spark.sql("""
SELECT
    (unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 3600 AS trip_length
FROM
    yellow_2024
ORDER BY
    trip_length DESC
""").show()

+------------------+
|       trip_length|
+------------------+
|162.61777777777777|
|           143.325|
|137.76055555555556|
|114.83472222222223|
| 89.89833333333333|
| 89.44611111111111|
| 70.29916666666666|
| 67.57333333333334|
| 66.06666666666666|
|           46.4225|
| 42.30888888888889|
| 38.47416666666667|
| 33.95111111111111|
| 26.29861111111111|
| 25.29138888888889|
|25.238333333333333|
|             24.47|
|23.996666666666666|
|23.995277777777776|
|23.994722222222222|
+------------------+
only showing top 20 rows



## Question 6

In [61]:
df_zone = spark.read \
    .option('header', 'true') \
    .csv('taxi_zone_lookup.csv')

In [63]:
df_zone.createOrReplaceTempView('taxi_zones')

### Join dataframes in spark

In [67]:
df_join = yellow_trips.join(F.broadcast(df_zone), yellow_trips.PULocationID == df_zone.LocationID, how='inner')

In [68]:
df_join.createOrReplaceTempView('joined')

In [87]:
spark.sql("""
SELECT
    COUNT(*) AS trip_count,
    Zone
FROM
    joined
GROUP BY
    Zone
ORDER BY
    COUNT(*) ASC
LIMIT 1
""").show()

+----------+--------------------+
|trip_count|                Zone|
+----------+--------------------+
|         1|Governor's Island...|
+----------+--------------------+



### Joining in Spark SQL using temp views

In [74]:
spark.sql("""
SELECT
    COUNT(*) AS trip_count,
    tz.Zone
FROM
    yellow_2024 yt
INNER JOIN
    taxi_zones tz
    ON yt.PULocationID = tz.LocationID
GROUP BY
    tz.Zone
ORDER BY
    trip_count ASC
LIMIT 1
""").show()

+----------+--------------------+
|trip_count|                Zone|
+----------+--------------------+
|         1|Governor's Island...|
+----------+--------------------+

